## ResNet50 Transfer Learning

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
cifar10 = keras.datasets.cifar10

(train_data, train_labels), (test_data, test_labels) = cifar10.load_data()

170500096/170498071 [==============================] - 6s 0us/step


In [3]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_labels))

In [4]:
IMG_SIZE = 160

In [5]:
def change_format(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/127.5) -1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label

In [6]:
train_dataset = train_dataset.map(change_format)
test_dataset = test_dataset.map(change_format)

In [7]:
train_dataset = train_dataset.shuffle(10000).batch(64)
test_dataset = test_dataset.batch(64)

### ResNet50

In [8]:
backbone = keras.applications.ResNet50(include_top=False,
                                       weights='imagenet',
                                       input_shape=(160, 160, 3))

94773248/94765736 [==============================] - 2s 0us/step


In [9]:
backbone.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 166, 166, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 80, 80, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 80, 80, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [10]:
backbone.trainable = False

In [11]:
model = keras.Sequential()
model.add(backbone)
model.add(keras.layers.GlobalAveragePooling2D())
model.add(keras.layers.Dense(10, activation='softmax'))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 5, 5, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                20490     
Total params: 23,608,202
Trainable params: 20,490
Non-trainable params: 23,587,712
_________________________________________________________________


In [13]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
model.fit(train_dataset, epochs=5)

Epoch 1/5
782/782 [==============================] - 45s 58ms/step - loss: 2.0976 - accuracy: 0.2528
Epoch 2/5
782/782 [==============================] - 45s 58ms/step - loss: 1.9304 - accuracy: 0.3326
Epoch 3/5
782/782 [==============================] - 46s 58ms/step - loss: 1.8591 - accuracy: 0.3604
Epoch 4/5
782/782 [==============================] - 45s 58ms/step - loss: 1.8135 - accuracy: 0.3779
Epoch 5/5
782/782 [==============================] - 45s 58ms/step - loss: 1.7788 - accuracy: 0.3863
